<a href="https://colab.research.google.com/github/SiyuanWuSFU/CM-T310-Artificial-Intelligence-Survey/blob/master/SVM_ipynb%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file is used to download Kaggle datasets into a personal Google Drive.
In order to use this, we need a kaggle.json file, which contains the API credentials.
To obtain a kaggle.json file, sign into your Kaggle account and go to your profile, then 'My Account'. Under the API section, click 'Create New API Token' to generate the kaggle.json file. Afterwards, save it in the same directory as this .ipynb file.

In [ ]:
!pip install tensorflow_addons
!pip install tensorflow
!pip install tensorflow --upgrade
!pip install keras --upgrade

In [56]:
!pip uninstall tensorflow_decision_forests


Found existing installation: tensorflow-decision-forests 1.5.0
Uninstalling tensorflow-decision-forests-1.5.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/tensorflow_decision_forests-1.5.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow_decision_forests/*
    /usr/local/lib/python3.10/dist-packages/yggdrasil_decision_forests/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-decision-forests-1.5.0


In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
dtypes={
    'elapsed_time':np.float32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    'room_coor_x':np.float32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'text':'category',
    'fqid':'category',
    'room_fqid':'category',
    'text_fqid':'category',
    'fullscreen':'category',
    'hq':'category',
    'music':'category',
    'level_group':'category'}



In [5]:
from google.colab import drive
drive.mount('/content/drive')

dataset_df = pd.read_csv('/content/drive/MyDrive/9417project/train.csv', dtype=dtypes)
labels = pd.read_csv('/content/drive/MyDrive/9417project/train_labels.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#Reduce Memory Usage
def reduce_memory_usage(df):

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024**2
    print("Memory usage became: ",mem_usg," MB")

    return df
dataset_df = reduce_memory_usage(dataset_df)

Memory usage of dataframe is 1529.83 MB
Memory usage became:  1053.3384094238281  MB


In [ ]:
print("Full train dataset shape is {}".format(dataset_df.shape))
dataset_df.head(5)

In [7]:
labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
labels.head(5)



,session_id,correct,session,q
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1


In [52]:
print("Full label dataset shape is {}".format(labels.shape))

Full label dataset shape is (424116, 4)


In [8]:
CATEGORICAL = ['event_name', 'name','fqid', 'room_fqid', 'text_fqid']
NUMERICAL = ['elapsed_time','level','page','room_coor_x', 'room_coor_y',
        'screen_coor_x', 'screen_coor_y', 'hover_duration']

In [9]:
def feature_engineer(dataset_df):
    dfs = []
    for c in CATEGORICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in NUMERICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('mean')
        dfs.append(tmp)
    for c in NUMERICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)
    dataset_df = pd.concat(dfs,axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id')
    return dataset_df

In [10]:
dataset_df = feature_engineer(dataset_df)
print("Full prepared dataset shape is {}".format(dataset_df.shape))

Full prepared dataset shape is (70686, 22)


In [ ]:
#dataset_df.head(25)
print(dataset_df.index.unique())

In [11]:
def split_dataset(dataset, test_ratio=0.20):
    USER_LIST = dataset.index.unique()
    split = int(len(USER_LIST) * (1 - 0.20))
    return dataset.loc[USER_LIST[:split]], dataset.loc[USER_LIST[split:]]

train_x, valid_x = split_dataset(dataset_df)
print("{} examples in training, {} examples in testing.".format(
    len(train_x), len(valid_x)))

56547 examples in training, 14139 examples in testing.


SVM model

In [12]:
from tensorflow import keras

from keras.layers import Activation, Dense
from keras.regularizers import l2
from keras.models import Sequential
from sklearn.metrics import f1_score
from sklearn.svm import SVC
model = SVC(probability=True)

In [13]:
# Fetch the unique list of user sessions in the validation dataset. We assigned
# `session_id` as the index of our feature engineered dataset. Hence fetching
# the unique values in the index column will give us a list of users in the
# validation set.
VALID_USER_LIST = valid_x.index.unique()

# Create a dataframe for storing the predictions of each question for all users
# in the validation set.
# For this, the required size of the data frame is:
# (no: of users in validation set  x no of questions).
# We will initialize all the predicted values in the data frame to zero.
# The dataframe's index column is the user `session_id`s.
prediction_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)

# Create an empty dictionary to store the models created for each question.
models = {}

# Create an empty dictionary to store the evaluation score for each question.
evaluation_dict ={}

In [14]:

for q_no in range(1,19):

    # Select level group for the question based on the q_no.
    if q_no<=3: grp = '0-4'
    elif q_no<=13: grp = '5-12'
    elif q_no<=22: grp = '13-22'
    print("### q_no", q_no, "grp", grp)


    # Filter the rows in the datasets based on the selected level group.
    train_df = train_x.loc[train_x.level_group == grp]
    train_users = train_df.index.values
    valid_df = valid_x.loc[valid_x.level_group == grp]
    valid_users = valid_df.index.values

    # Drop the 'correct' and 'level_group' columns from the features
    train_df = train_df.drop(columns=["level_group"])
    valid_df = valid_df.drop(columns=["level_group"])

    # Select the labels for the related q_no.
    train_labels = labels.loc[labels.q==q_no].set_index('session').loc[train_users]
    valid_labels = labels.loc[labels.q==q_no].set_index('session').loc[valid_users]
    train_labels = train_labels['correct']
    valid_labels = valid_labels['correct']
    print(train_labels)
    #train_df = np.asarray(train_df).astype(np.float32)
    #train_labels = np.asarray(train_labels).astype(np.float32)

    # Add the label to the filtered datasets.
    # train_dataset = tf.data.Dataset.from_tensor_slices((train_df, train_labels))
    # test_dataset = tf.data.Dataset.from_tensor_slices((valid_df, valid_labels))

    # train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size)
    # test_dataset = test_dataset.batch(batch_size)

    # SVM_model = keras.Sequential()
    # SVM_model.add(Dense(64, activation='relu'))
    # SVM_model.add(Dense(1, kernel_regularizer=l2(0.01)))
    # SVM_model.add(layers.Activation('linear'))

    # SVM_model.compile(loss='hinge',
    #           optimizer='adam',
    #           metrics=['accuracy'])
    # Train the model.
    print(train_df.shape)
    #SVM_model.fit(train_dataset,epochs=50)
    model.fit(train_df,train_labels)
    model.score(valid_df,valid_labels)

    valid_pred = model.predict(valid_df)
    valid_pred_proba = model.predict_proba(valid_df)[:, 1]
    # Store the model

    #models[f'{grp}_{q_no}'] = SVM_model
    # Evaluate the trained model on the validation dataset and store the
    # evaluation accuracy in the `evaluation_dict`.
    #loss, accuracy = SVM_model.evaluate(test_dataset,return_dict=True)

    #evaluation_dict[q_no] = accuracy

    # Use the trained model to make predictions on the validation dataset and
    # store the predicted values in the `prediction_df` dataframe.
    #predict = SVM_model.predict(test_dataset)
    prediction_df.loc[valid_users, q_no-1] = valid_pred_proba.flatten()

### q_no 1 grp 0-4
session
20090312431273200    1
20090312433251036    0
20090312455206810    1
20090313091715820    0
20090313571836404    1
                    ..
22000319034806750    0
22000319082126604    1
22000319092201970    1
22000319145510864    1
22000319365553870    1
Name: correct, Length: 18849, dtype: int64
(18849, 21)
[0.72627703 0.72627834 0.72627814 ... 0.72627567 0.72627817 0.72627523]
f1_score: 0.8454189122978932
### q_no 2 grp 0-4
session
20090312431273200    1
20090312433251036    1
20090312455206810    1
20090313091715820    1
20090313571836404    1
                    ..
22000319034806750    1
22000319082126604    1
22000319092201970    1
22000319145510864    1
22000319365553870    1
Name: correct, Length: 18849, dtype: int64
(18849, 21)
[0.97966906 0.97966952 0.97966937 ... 0.97966766 0.97966952 0.9796675 ]
f1_score: 0.9876490172913758
### q_no 3 grp 0-4
session
20090312431273200    1
20090312433251036    1
20090312455206810    1
20090313091715820    1
200903135

In [22]:
print(prediction_df)
print(valid_users)

                         0         1         2         3         4         5   \
session_id                                                                      
22000320020067784  0.726277  0.979669  0.933838  0.800185  0.552531  0.773154   
22000321083750010  0.726278  0.979670  0.933834  0.800179  0.553719  0.773155   
22000401381351532  0.726278  0.979669  0.933834  0.800178  0.553975  0.773155   
22000407142860316  0.726277  0.979669  0.933836  0.800194  0.552261  0.773154   
22000407572357990  0.726278  0.979669  0.933834  0.800254  0.523193  0.773153   
...                     ...       ...       ...       ...       ...       ...   
22100215342220508  0.726274  0.979666  0.933845  0.800229  0.544883  0.773154   
22100215460321130  0.726276  0.979668  0.933839  0.800207  0.548451  0.773154   
22100217104993650  0.726276  0.979668  0.933841  0.800206  0.550138  0.773154   
22100219442786200  0.726278  0.979670  0.933834  0.800181  0.553326  0.773154   
22100221145014656  0.726275 

In [ ]:
for i in valid_pred:
  print(i)

In [123]:

valid_labels_correct = train_labels
#valid_labels_incorrect = labels.loc[(labels.q==1) & (labels.correct == 0)]
print(valid_labels_correct)
#print(valid_labels_incorrect)





session
20090312431273200    1
20090312433251036    0
20090312455206810    1
20090313091715820    0
20090313571836404    1
                    ..
22000319034806750    0
22000319082126604    1
22000319092201970    1
22000319145510864    1
22000319365553870    1
Name: correct, Length: 18849, dtype: int64


In [21]:
print(model.score(valid_df,valid_labels))

0.9516231699554424


In [19]:
for name, value in evaluation_dict.items():
  print(f"question {name}: accuracy {value:.4f}")

print("\nAverage accuracy", sum(evaluation_dict.values())/18)


Average accuracy 0.0


In [27]:
true_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)
for i in range(18):
    # Get the true labels.
    tmp = labels.loc[labels.q == i+1].set_index('session').loc[VALID_USER_LIST]
    true_df[i] = tmp.correct.values

max_score = 0; best_threshold = 0

# Loop through threshold values from 0.4 to 0.8 and select the threshold with
# the highest `F1 score`.
# using uniform threashold = 0.5 for the baseline


# for threshold in np.arange(0.4,0.8,0.01):
#     metric = tf.keras.metrics.F1Score(average="macro",threshold=threshold)
#     y_true = tf.one_hot(true_df.values.reshape((-1)), depth=2)
#     y_pred = tf.one_hot((prediction_df.values.reshape((-1))>threshold).astype('int'), depth=2)
#     metric.update_state(y_true, y_pred)
#     f1_score = metric.result().numpy()
#     if f1_score > max_score:
#         max_score = f1_score
#         best_threshold = threshold
metric = tf.keras.metrics.F1Score(average="macro",threshold=0.5)
y_true = tf.one_hot(true_df.values.reshape((-1)), depth=2)
y_pred = tf.one_hot((prediction_df.values.reshape((-1))>0.5).astype('int'), depth=2)
metric.update_state(y_true, y_pred)
f1_score = metric.result().numpy()

#print("threshold ", best_threshold, "\tF1 score ", max_score)

print("threshold ", 0.5, "\tF1 score ", f1_score)

threshold  0.6200000000000002 	F1 score  0.6503212


In [ ]:
from google.colab import drive
drive.mount('/content/drive')